In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Import train datastet
train = pd.read_csv('train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test=pd.read_csv('test.csv')
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Checking missing values
def find_missing(train,test):
    count_missing_train=train.isnull().sum().values
    count_missing_test=train.isnull().sum().values
    total_train=train.shape[0]
    total_test=test.shape[0]
    ratio_missing_train=count_missing_train/total_train*100
    ratio_missing_test=count_missing_test/total_test*100
    return pd.DataFrame({'Missing_train':count_missing_train,'Missing_Ratio_Train':ratio_missing_train,
                        'Missing_test':count_missing_test,'Missing_Ratio_Test':ratio_missing_test},
                       index=train.columns)
df_missing=find_missing(train.drop(columns='label',axis=1),test)
df_missing=df_missing[df_missing['Missing_Ratio_Train']>0].sort_values(by='Missing_Ratio_Train',ascending=False)

In [5]:
df_missing.head()

,Missing_train,Missing_Ratio_Train,Missing_test,Missing_Ratio_Test


In [6]:
# There is no missing values in train and test sets

In [7]:
# Splitting to target
train_target=train.iloc[:,0].values

In [8]:
# Normalization
train=train.drop(columns='label')/255.0
test=test/255.0
# Reshape
train=train.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

In [9]:
# Encoding the target
train_target=pd.get_dummies(train_target)

In [10]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train,train_target,test_size=0.2,random_state=0)

In [11]:
# Building the CNN
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the CNN
classifier=Sequential()
# Convolution
classifier.add(Convolution2D(32,5,5,input_shape=(28,28,1),activation='relu'))
classifier.add(Convolution2D(32,5,5,input_shape=(28,28,1),activation='relu'))
# Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

# Another Convolution
classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(Convolution2D(64,3,3,activation='relu'))
#Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

# Flatten
classifier.add(Flatten())

# Full Connection
classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(output_dim=10,activation='sigmoid'))

# Compiling the CNN
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\limo\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\Users\limo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(28, 28, 1..., activation="relu")`
  if sys.path[0] == '':
C:\Users\limo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(28, 28, 1..., activation="relu")`
  del sys.path[0]
C:\Users\limo\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Users\limo\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), ac

In [12]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [13]:
train_datagen.fit(train)

In [19]:
classifier.fit_generator(
train_datagen.flow(train,train_target,batch_size=84),nb_epoch=50,
    validation_data=(x_train,y_train),verbose=2,samples_per_epoch=train.shape[0])

C:\Users\limo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\limo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., verbose=2, steps_per_epoch=500, epochs=50)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/50
 - 1391s - loss: 0.0635 - acc: 0.9809 - val_loss: 0.0241 - val_acc: 0.9925
Epoch 2/50
 - 1372s - loss: 0.0653 - acc: 0.9812 - val_loss: 0.0222 - val_acc: 0.9934
Epoch 3/50
 - 1369s - loss: 0.0626 - acc: 0.9818 - val_loss: 0.0225 - val_acc: 0.9928
Epoch 4/50
 - 1391s - loss: 0.0618 - acc: 0.9815 - val_loss: 0.0211 - val_acc: 0.9937
Epoch 5/50
 - 1393s - loss: 0.0620 - acc: 0.9815 - val_loss: 0.0232 - val_acc: 0.9931
Epoch 6/50
 - 1406s - loss: 0.0651 - acc: 0.9806 - val_loss: 0.0220 - val_acc: 0.9930
Epoch 7/50
 - 1357s - loss: 0.0583 - acc: 0.9826 - val_loss: 0.0195 - val_acc: 0.9940
Epoch 8/50
 - 1397s - loss: 0.0593 - acc: 0.9823 - val_loss: 0.0249 - val_acc: 0.9922
Epoch 9/50
 - 1394s - loss: 0.0593 - acc: 0.9830 - val_loss: 0.0198 - val_acc: 0.9937
Epoch 10/50
 - 1387s - loss: 0.0580 - acc: 0.9831 - val_loss: 0.0206 - val_acc: 0.9936
Epoch 11/50
 - 1382s - loss: 0.0560 - acc: 0.9825 - val_loss: 0.0253 - val_acc: 0.9919
Epoch 12/50
 - 1385s - loss: 0.0609 - acc: 0.9822 - 

In [20]:
predictions=classifier.predict(test)

In [21]:
predictions=np.argmax(predictions,axis=1)

In [22]:
predictions=pd.Series(predictions)

In [23]:
submission=pd.DataFrame({'ImageId':range(1,28001)})
submission['label']=predictions
submission.to_csv('submission.csv',index=False)